<a href="https://colab.research.google.com/github/AINACIOMENDONCA/MYDASHBOARD/blob/master/API_IBGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Author:  Andrea Mendonca Inacio
# 28/09/2022
# Extração de DADOS (WebScraping) via APIs do IBGE
# APIs: Municípios por ESTADO (UF) e População por UF

# API consulta Unidades Federativas brasileiras (UF)
# https://servicodados.ibge.gov.br/api/v1/localidades/estados

# API consulta População de cada UF
# https://servicodados.ibge.gov.br/api/v1/projecoes/populacao/{ID_UF}

# API consulta Localidades (Municípios) de cada UF
# https://servicodados.ibge.gov.br/api/v1/localidades/estados/{UF}/municipios

import requests as rq
import json
import pandas as pd
import sqlite3
import plotly.express as px

In [ ]:
# Criação do DATABASE para importação do tabela gerada na extração do dados do IBGE
db = sqlite3.connect ('IBGE_API.db')

In [ ]:
# Executa API usando biblioteca Requests. Carrega arquivo JSON
RESPONSE = rq.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados")
RETORNO_REQ = json.loads(RESPONSE.text)

# Faz a leitura do JSON
# Atribui variáveis de cada Estado em uma lista
# Cria Lista de Listas com as informações de cada Estado. 
# Importante: ID_UF é uma convenção (código que representa cada Estado Brasileiro),
#             é usado por vários sistemas.

UF = []
for CHAVE in RETORNO_REQ:
  
  UF_LISTA = [CHAVE.get('id'), CHAVE.get('nome'), CHAVE.get('sigla')]
  # Criando Lista de Listas principal para UF
  UF.append(UF_LISTA)

print(UF)

[[11, 'Rondônia', 'RO'], [12, 'Acre', 'AC'], [13, 'Amazonas', 'AM'], [14, 'Roraima', 'RR'], [15, 'Pará', 'PA'], [16, 'Amapá', 'AP'], [17, 'Tocantins', 'TO'], [21, 'Maranhão', 'MA'], [22, 'Piauí', 'PI'], [23, 'Ceará', 'CE'], [24, 'Rio Grande do Norte', 'RN'], [25, 'Paraíba', 'PB'], [26, 'Pernambuco', 'PE'], [27, 'Alagoas', 'AL'], [28, 'Sergipe', 'SE'], [29, 'Bahia', 'BA'], [31, 'Minas Gerais', 'MG'], [32, 'Espírito Santo', 'ES'], [33, 'Rio de Janeiro', 'RJ'], [35, 'São Paulo', 'SP'], [41, 'Paraná', 'PR'], [42, 'Santa Catarina', 'SC'], [43, 'Rio Grande do Sul', 'RS'], [50, 'Mato Grosso do Sul', 'MS'], [51, 'Mato Grosso', 'MT'], [52, 'Goiás', 'GO'], [53, 'Distrito Federal', 'DF']]


In [ ]:
# https://servicodados.ibge.gov.br/api/v1/projecoes/populacao/{ID_UF}
# https://servicodados.ibge.gov.br/api/v1/localidades/estados/{UF}/municipios

MUNICIPIOS = []  # Lista que será usada para criação do Dataframe para Municípios versus Estado
POVO = []  # Lista que será usada para criação do Dataframe para População versus Estado

for i in UF:
  ID_UF= i[0]  # Posição 0 de cada sublista da lista UF
  NOME_UF = i[1] # Posição 1 de cada sublista da lista UF
  SIGLA_UF= i[2] # Posição 2 de cada sublista da lista UF

  # Utiliza IDs das Unidades Federativas (para execução da API que traz a população de cada UF)
  
  POPULACAO = rq.get(f"https://servicodados.ibge.gov.br/api/v1/projecoes/populacao/{ID_UF}")
  # Usa variavel auxiliar para acessar subchave do JSON
  AUX = POPULACAO.json()['projecao']
  ESTIMATIVA_POP = AUX.get('populacao')

  POVO_LISTA = [NOME_UF,ESTIMATIVA_POP]
  # Criando Lista de Listas principal para POVO (populacao de cada Estado)
  POVO.append(POVO_LISTA)
  #print ("IBGE estima que a população da Unidade Federativa Brasileira: ", NOME_UF, "-", SIGLA_UF, " é de ", ESTIMATIVA_POP )

  # Utiliza SIGLA_UF para execução da API que traz os nomes dos municípios de cada UF.
  # Esta API também traz informações da região onde está localizado o Município.

  CIDADES = rq.get(f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{SIGLA_UF}/municipios")
  JSON_CIDADES = json.loads(CIDADES.text)

  # Para cada posição do JSON, carrega variáveis pertinentes para a tabela que será criada.
  for city in JSON_CIDADES:
    MUNICIPIO = city.get('nome')  ### Atributo MUNICIPIO
    MICRORREGIAO = city.get('microrregiao')
    MESORREGIAO = MICRORREGIAO.get('mesorregiao')
    NOME_MESORREGIAO = MESORREGIAO.get('nome')  ### Atributo MESORREGIAO
    LISTA = [MUNICIPIO,NOME_MESORREGIAO,NOME_UF,SIGLA_UF,ID_UF,ESTIMATIVA_POP]
    MUNICIPIOS.append (LISTA)
  

In [ ]:
# Cria dataframe para Municípios e informaçoes sobre Estado em que está localizado.
DF_MUNICIPIOS = pd.DataFrame (MUNICIPIOS, columns=['Municipio','Regiao','Estado','UF','ID_UF','Populacao_UF'])

DF_MUNICIPIOS.describe

# Cria dataframe para População versus Estado
DF_POVO= pd.DataFrame (POVO, columns=['UF','POPULACAO'])
#DF_POVO.describe


In [ ]:
fig = px.line(DF_POVO, x = 'UF', y='POPULACAO')
fig.show()

In [ ]:
QTD_MUNICIPIOS_AUX = DF_MUNICIPIOS.groupby(['UF'])['Municipio'].count().sort_values()
print(QTD_MUNICIPIOS_AUX)
fig = px.bar(QTD_MUNICIPIOS_AUX)
fig.show()

UF
DF      1
RR     15
AP     16
AC     22
RO     52
AM     62
SE     75
ES     78
MS     79
RJ     92
AL    102
TO    139
MT    141
PA    144
RN    167
CE    184
PE    185
MA    217
PB    223
PI    224
GO    246
SC    295
PR    399
BA    417
RS    497
SP    645
MG    853
Name: Municipio, dtype: int64


In [ ]:
DF_MUNICIPIOS.to_sql ('MUNICIPIOS_BRASILEIROS', db, if_exists= "replace", index = False)
 

In [ ]:
# Executa Query pelo SQLite

query = ''' select * from MUNICIPIOS_BRASILEIROS '''
df = pd.read_sql_query (query,db)
df.head()

,Municipio,Regiao,Estado,UF,ID_UF,Populacao_UF
0,Alta Floresta D'Oeste,Leste Rondoniense,Rondônia,RO,11,1837876
1,Ariquemes,Leste Rondoniense,Rondônia,RO,11,1837876
2,Cabixi,Leste Rondoniense,Rondônia,RO,11,1837876
3,Cacoal,Leste Rondoniense,Rondônia,RO,11,1837876
4,Cerejeiras,Leste Rondoniense,Rondônia,RO,11,1837876


In [ ]:
# Executa Query para Municipios do Estado de São Paulo e região de Campinas

query = ''' select Municipio from MUNICIPIOS_BRASILEIROS \
        where Estado LIKE "São Paulo" AND regiao LIKE "Campinas" '''

df = pd.read_sql_query (query,db)
df

,Municipio
0,Aguaí
1,Águas da Prata
2,Águas de Lindóia
3,Americana
4,Amparo
5,Artur Nogueira
6,Caconde
7,Campinas
8,Casa Branca
9,Cosmópolis


In [ ]:
DF_POVO.to_sql ('POPULACAO', db, if_exists= "replace", index = False)
DF_POVO

,UF,POPULACAO
0,Rondônia,1837876
1,Acre,922152
2,Amazonas,4346362
3,Roraima,665632
4,Pará,8882579
5,Amapá,897109
6,Tocantins,1628110
7,Maranhão,7199679
8,Piauí,3297171
9,Ceará,9304699


In [ ]:
# Executa Query para total da população brasileira pela soma da populacao dos Estados

query = ''' select sum (POPULACAO) from POPULACAO '''

df = pd.read_sql_query (query,db)
df

,sum (POPULACAO)
0,215167488


In [ ]:
# Executa Query para descobrir qual o Estado tem a menor populacao

query1 = ''' select UF, min(POPULACAO) as menor_populacao \
            from POPULACAO '''

df = pd.read_sql_query (query1,db)
df


,UF,menor_populacao
0,Roraima,665632


In [ ]:
# Executa Query para descobrir qual o Estado tem a maior populacao

query2 = ''' select UF, max(POPULACAO) as maior_populacao \
            from POPULACAO '''

df = pd.read_sql_query (query2,db)
df

,UF,maior_populacao
0,São Paulo,47078488


In [ ]:
# Executa Query para descobrir quantos municípios existem no Estado de Roraima

query = ''' select count(Municipio) \
            from MUNICIPIOS_BRASILEIROS \
            where UF LIKE "RR" '''

df = pd.read_sql_query (query,db)
df

,count(Municipio)
0,15


In [ ]:
# Executa Query para descobrir quantos municípios existem no Estado de São Paulo

query = ''' select count(Municipio) \
            from MUNICIPIOS_BRASILEIROS \
            where UF LIKE "SP" '''

df = pd.read_sql_query (query,db)
df

,count(Municipio)
0,645
